In [1]:
import pandas as pd
import plotly as plt
import matplotlib as mplt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [21]:
def plot_barrier(rs,kdb):
    data = [go.Scatter(x=rs['barrier'],
                       y = rs['force call saddle']/kdb['force call saddle'],
                       mode = 'markers',
                       marker = dict(size = 30, line = dict(width = 2)))]
    fig = go.Figure(data)
    fig.update_layout( 
                  showlegend = False,
                  xaxis = dict(title_text = '<b>Barrier<b>', title_font = {'size':40}),
                  yaxis = dict(title_text = '<b>Ratio RS : KDB<b>', title_font = {'size':40}),
                  template = 'plotly_white',
                  width=3000, height=1500)
    fig.update_xaxes(ticks='outside', showline=True, linewidth=1, linecolor='black', tickfont=dict(size=30))
    fig.update_yaxes(ticks='outside', showline=True, linewidth=1, linecolor='black', tickfont=dict(size=30))
    fig.show()

In [ ]:
def compare_barrier(rs,kdb):
    data = []
    table = [rs,kdb]
    name = ['Random Searching', 'KDB']
    color = ['purple','orange']
    
    for i in range(len(table)):
        saddle = [j for j in range(len(table[i]))]
        data.append(go.Bar(x=saddle, y = table[i]['force call saddle'], name = name[i], marker_color = color[i]))
    
    fig = go.Figure(data)
    fig.update_layout(title = '<b> Force call saddle comparison between Random Searching and KDB <b>',
                  legend_title = '<b> Method <b>', 
                  xaxis_title = '<b>Saddle #<b>',
                  yaxis_title = '<b>Force call<b>',
                  xaxis_type = 'category')
    
    fig.show()

In [3]:
def path(n):
    path_rs = 'ransearch/result' + str(n) + '.csv'
    path_kdb = 'onlykdb/result' + str(n) + '.csv'
    return path_rs, path_kdb

def table(n):
    tab_rs = 'rs' + str(n)
    tab_kdb = 'kdb' + str(n)
    return tab_rs, tab_kdb

def query(table):
    query = 'SELECT rs.barrier FROM ' + str(table[0]) + ' rs JOIN ' + str(table[1]) + ' kdb ON rs.barrier = kdb.barrier'
    return query

def find_same_barrier(query):
    lst = engine.execute(query).fetchall()
    same_barrier = []
    for n in lst:
        same_barrier.append(n[0])
    return same_barrier

def find_barrier(i):
    rs = pd.read_csv(path(i)[0])
    kdb = pd.read_csv(path(i)[1])
    rs_tab, kdb_tab = table(i)
    rs.to_sql(rs_tab, con = engine)
    kdb.to_sql(kdb_tab, con = engine)
    same_barrier = find_same_barrier(query(table(i)))
    return same_barrier

In [4]:
def combine_table(n):
    rs_combine = pd.DataFrame()
    kdb_combine = pd.DataFrame()
    for i in range(n):
        rs = pd.read_csv(path(i)[0])
        kdb = pd.read_csv(path(i)[1])
        b_lst = find_barrier(i)
        for b in b_lst:
            row_rs = rs.loc[rs['barrier']==b]
            rs_combine = rs_combine.append(row_rs,ignore_index=True)
            row_kdb = kdb.loc[kdb['barrier']==b]
            kdb_combine = kdb_combine.append(row_kdb,ignore_index=True)
    
    full_rs = rs_combine[['barrier','force call saddle']]
    full_kdb = kdb_combine[['barrier','force call saddle']]
    full_rs.index.names = ['saddle']
    full_kdb.index.names = ['saddle']
    full_rs = full_rs.sort_values(by='barrier')
    full_kdb = full_kdb.sort_values(by='barrier')
    return full_rs, full_kdb

In [5]:
rs, kdb = combine_table(10)

In [22]:
plot_barrier(rs,kdb)